In [769]:
#Suzan Iloglu, May 21,2020
import gurobipy as gp
import csv
from gurobipy import GRB
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200

In [770]:
from IPython.display import Image

## Social Vulnerability Index for Mapping Community Health Worker Need
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a>

### I. Importing SVI data and computing state percentile ranking
The CDC uses both a USA-wide percentile ranking and a state by state percentile ranking. For our project given that funding is likely going to be managed at a state level using a state percentile ranking makes the most sense and will be most sensitive to regional socioeconomic differences. 

In [771]:
## import svi data by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [772]:
## replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
State = svi_county.STATE.unique().tolist()

In [773]:
# Create a seperate dictionary for the components of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

In [774]:
# Create a dictionary for minorities per county
Minority_county = E_MINRTY


In [775]:
# Read the medicaid demand data
df_m = pd.read_csv("Data/Medicaid_Demand.csv")


In [776]:
# Read the Unemployment data
df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
df_unemp = df_unemp.fillna(0)


In [777]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


In [778]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [779]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed))

# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be
# Med patient number divided by 55
CHW_demand = dict(zip(df_m.FIPS, df_m.Med_Demand/55))


In [781]:
#### data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [782]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [783]:
import requests
import io

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))
covid.head(10)
#covid.drop(columns = ['deaths'])

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0
5,2020-01-25,Orange,California,6059.0,1,0
6,2020-01-25,Cook,Illinois,17031.0,1,0
7,2020-01-25,Snohomish,Washington,53061.0,1,0
8,2020-01-26,Maricopa,Arizona,4013.0,1,0
9,2020-01-26,Los Angeles,California,6037.0,1,0


In [818]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/boro/boroughs-case-hosp-death.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))
#covid_ny.head(10)



In [819]:
Kings = covid_ny[['DATE_OF_INTEREST', 'BK_CASE_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'DATE_OF_INTEREST': 'date'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = '36047'
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [820]:
Bronx = covid_ny[['DATE_OF_INTEREST', 'BX_CASE_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'DATE_OF_INTEREST': 'date'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = '36005'
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [821]:
Manhattan = covid_ny[['DATE_OF_INTEREST', 'MN_CASE_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'DATE_OF_INTEREST': 'date'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = '36061'
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [822]:
Queens = covid_ny[['DATE_OF_INTEREST', 'QN_CASE_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'DATE_OF_INTEREST': 'date'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = '36081'
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [789]:
Richmond = covid_ny[['DATE_OF_INTEREST', 'SI_CASE_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'DATE_OF_INTEREST': 'date'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = '36085'

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

,date,cases,state,county,fips
0,02/29/2020,0,New York,Richmond,36085
1,03/01/2020,0,New York,Richmond,36085
2,03/02/2020,0,New York,Richmond,36085
3,03/03/2020,0,New York,Richmond,36085
4,03/04/2020,0,New York,Richmond,36085


In [790]:
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0.0
1,2020-01-22,Snohomish,Washington,53061,1,0.0
2,2020-01-23,Snohomish,Washington,53061,1,0.0
3,2020-01-24,Cook,Illinois,17031,1,0.0
4,2020-01-24,Snohomish,Washington,53061,1,0.0
5,2020-01-25,Orange,California,6059,1,0.0
6,2020-01-25,Cook,Illinois,17031,1,0.0
7,2020-01-25,Snohomish,Washington,53061,1,0.0
8,2020-01-26,Maricopa,Arizona,4013,1,0.0
9,2020-01-26,Los Angeles,California,6037,1,0.0


In [791]:

covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)

In [792]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [793]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [794]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
181,08/28/2020,Bronx,New York,36005,0,NaN,2020-08-28,-1 days,0.0
181,08/28/2020,Richmond,New York,36085,0,NaN,2020-08-28,-1 days,0.0
467535,2020-08-25,Rawlins,Kansas,20153,1,0.0,2020-08-25,-4 days,0.0
439182,2020-08-16,Arthur,Nebraska,31005,1,0.0,2020-08-16,-13 days,0.0
477252,2020-08-28,Wallace,Kansas,20199,1,0.0,2020-08-28,-1 days,0.0
477229,2020-08-28,Rawlins,Kansas,20153,1,0.0,2020-08-28,-1 days,0.0
462928,2020-08-23,Sterling,Texas,48431,1,0.0,2020-08-23,-6 days,0.0
446788,2020-08-18,Sterling,Texas,48431,1,0.0,2020-08-18,-11 days,0.0
469386,2020-08-25,Sterling,Texas,48431,1,0.0,2020-08-25,-4 days,0.0
477177,2020-08-28,Elk,Kansas,20049,1,0.0,2020-08-28,-1 days,0.0


In [795]:
## group by to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases','deaths'].sum()

In [796]:
#svi_county.head(5)
#covid_last14_stats.head(5)
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [797]:

County_covid_death = dict(zip(covid_last14_stats.FIPS, covid_last14_stats.deaths))


In [798]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.new_cases))
County_demand = {}

for j in SVI_county:
    if j in covid_cases_county_ny_times:
        County_demand[j] = covid_cases_county_ny_times[j]   
    else:
        County_demand[j] = 0


In [799]:
############################################################################
############ END of demand calculation #####################################

In [800]:
df = covid_last14_stats
df.head(10)
df['fips'] = df['fips'].astype(int)

In [801]:


# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = [k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [802]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [803]:
Number_of_county_per_State = {}

count = 0
for s in State:
    for j in location:
        if (j,s) in cartesian_pro_county_state:
            count += 1
    Number_of_county_per_State[s] = count
    count = 0
    


In [804]:

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(dict_1[j] for (j,s) in cartesian_pro_county_state.select('*', s))  
    return state_dict



In [805]:
def Proportional(county_level, state_level):
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6:
            prop[j] = (county_level[j]/state_level[s])    
        else:
            prop[j] = 0
                
    return prop

In [806]:
# These dicts are the components of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)


E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )



In [807]:
from collections import Counter

SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


In [809]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6:
            prop_allocate[j,s] = (county_level[j]/state_level[s])*state_budget[s]
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [810]:
Medicaid_demand_state = total_state(Medicaid_demand) #sum(Medicaid_demand[j] for j in location if (j,s) in cartesian_pro_county_state)
Covid_state = total_state(County_demand) #sum(county_demand[j] for j in location if (j,s) in cartesian_pro_county_state)
SVI_state = total_state(SVI_county) #sum(SVI_county[j] for j in location if (j,s) in cartesian_pro_county_state)
YPLL_state = total_state(YPLL) #sum(YPLL[j] for j in location if (j,s) in cartesian_pro_county_state if j in YPLL)
Unemployment_state = total_state(Unemployment)  

In [811]:

Federal_budget_CHW = 1000000
Medicaid_budget_state = {}
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)


for s in State:
    Medicaid_budget_state[s] = (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW
    
#print (Medicaid_budget_state)
#print (sum(Medicaid_budget_state[s] for s in State))

In [812]:
# Calculte the population per state
State_pop = {}
for s in State:   
    State_pop[s] = sum(population_county[j] for j in location if (j,s) in cartesian_pro_county_state)
    

In [813]:
 
#Normalize

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

In [814]:
SVI_med = {j: (Proportional(SVI_county, total_state(SVI_county))[j] + Proportional(Medicaid_demand, total_state(Medicaid_demand))[j])/2 for j in location }

YPLL_med = {j: (Proportional(YPLL, total_state(YPLL))[j] + Proportional(Medicaid_demand, total_state(Medicaid_demand))[j])/2 for j in location  }

Unemployment_med = {j:  (Proportional(Unemployment, total_state(Unemployment))[j] + Proportional(Medicaid_demand, total_state(Medicaid_demand))[j])/2 for j in location }

Covid_med = {j: (Proportional(County_demand, total_state(County_demand))[j] + Proportional(Medicaid_demand, total_state(Medicaid_demand))[j])/2 for j in location}

Covid_capita = {j: 100000*(County_demand[j]/population_county[j]) for j in location}

Covid_capita_med = {j: 100000*(((Proportional(County_demand, total_state(County_demand))[j] + Proportional(Medicaid_demand, total_state(Medicaid_demand))[j]) / 2)/population_county[j]) for j in location}

Covid_death_capita_med = {j: 100000*(((Proportional(County_covid_death, total_state(County_covid_death))[j] + Proportional(Medicaid_demand, total_state(Medicaid_demand))[j])/2)/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}


Covid_med_state = total_state(Covid_med)

SVI_med_state = total_state(SVI_med)

YPLL_med_state = total_state(YPLL_med)

Unemployment_med_state = total_state(Unemployment_med)

  
Total_covid_cap = total_state(Covid_capita) 

Total_covid_cap_med = total_state(Covid_capita_med) 

Total_covid_death_cap = total_state(Covid_death_capita) 

Total_covid_death_cap_med = total_state(Covid_death_capita_med) 

Total_medicaid_cap = total_state(Medicaid_capita) 


In [815]:

Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

Proportional_to_covid_death_cap_med = Proportional_allocation(Covid_death_capita_med, Total_covid_death_cap_med,Medicaid_budget_state)  

Proportional_to_medicaid = Proportional_allocation(Medicaid_demand,Medicaid_demand_state,Medicaid_budget_state )

Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

Proportional_to_covid = Proportional_allocation(County_demand, Covid_state, Medicaid_budget_state)

Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)

Proportional_to_covid_med = Proportional_allocation(Covid_med, Covid_med_state, Medicaid_budget_state)
  
Proportional_to_SVI_med  = Proportional_allocation(SVI_med, SVI_med_state, Medicaid_budget_state)

Proportional_to_covid_capita_med = Proportional_allocation(Covid_capita_med, Total_covid_cap_med, Medicaid_budget_state)

Proportional_to_unemployment_med = Proportional_allocation(Unemployment_med, Unemployment_med_state, Medicaid_budget_state)

Proportional_to_YPLL_med = Proportional_allocation(YPLL_med, YPLL_med_state, Medicaid_budget_state)



In [816]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [817]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Covid_capita",  "Covid_death_capita"
              , "Covid_med", "SVI_med", "YPLL_med", "Covid_capita_med","Covid_death_capita_med", "Unemployment_med", ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Normalized_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(), Medicaid_demand.values(), normalize(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(), Medicaid_capita.values(), normalize(Medicaid_capita).values()
                   , Proportional_to_covid.values(),  County_demand.values(), normalize(County_demand).values()
                   , Proportional_to_SVI.values(),SVI_county.values(), normalize(SVI_county).values()
                   , Proportional_to_YPLL.values(), YPLL.values(), normalize(YPLL).values()
                   , Proportional_to_unemployment.values(), Unemployment.values(),  normalize(Unemployment).values()
                   , Proportional_to_covid_capita.values(), Covid_capita.values(), normalize(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(), Covid_death_capita.values(), normalize(Covid_death_capita).values()
                   , Proportional_to_covid_med.values(), Covid_med.values(), normalize(Covid_med).values()
                   , Proportional_to_SVI_med.values(), SVI_med.values(), normalize(SVI_med).values()
                   , Proportional_to_YPLL_med.values(), YPLL_med.values(), normalize(YPLL_med).values()
                   , Proportional_to_covid_capita_med.values(), Covid_capita_med.values(), normalize(Covid_capita_med).values()
                   , Proportional_to_covid_death_cap_med.values(), Covid_death_capita_med.values(), normalize(Covid_death_capita_med).values()
                   , Proportional_to_unemployment_med.values(), Unemployment_med.values(), normalize(Unemployment_med).values() ):                    
       
        writer.writerow(row)